In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
storage_account_name = "<Storage account name>"
storage_account_key = "<Storage account key>"
container = "<Container name>"

In [3]:
spark.conf.set("fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name), storage_account_key)

In [4]:
data = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("delimiter", ",") \
  .csv("wasbs://{0}@{1}.blob.core.windows.net/housing.csv".format(container, storage_account_name))

data.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
 -122.23| 37.88| 41.0| 880.0| 129.0| 322.0| 126.0| 8.3252| 452600.0| NEAR BAY|
 -122.22| 37.86| 21.0| 7099.0| 1106.0| 2401.0| 1138.0| 8.3014| 358500.0| NEAR BAY|
 -122.24| 37.85| 52.0| 1467.0| 190.0| 496.0| 177.0| 7.2574| 352100.0| NEAR BAY|
 -122.25| 37.85| 52.0| 1274.0| 235.0| 558.0| 219.0| 5.6431| 341300.0| NEAR BAY|
 -122.25| 37.85| 52.0| 1627.0| 280.0| 565.0| 259.0| 3.8462| 342200.0| NEAR BAY|
 -122.25| 37.85| 52.0| 919.0| 213.0| 413.0| 193.0| 4.0368| 269700.0| NEAR BAY|
 -122.25| 37.84| 52.0| 2535.0| 489.0| 1094.0| 514.0| 3.6591| 299200.0| NEAR BAY|
 -122.25| 37.84| 52.0| 3104.0| 687.0| 1157.0| 647.0| 3.12| 241400.0| NEAR BAY|
 -122.26| 37.84| 42.0| 2555.0| 665.0| 1206.0| 595.0| 2.0804| 226700.0| NEAR BAY|
 -122.25| 37.84| 52.0| 3549.0| 707.0| 1551.0| 714.0| 3.6912| 261100.0| NEAR BAY|
 -122.26| 37.85| 52.0| 2202.0| 434.0| 910.0| 402.0| 3.2031| 281500.0| NEAR BAY|
 -122.26| 37.85| 52.0| 3503.0| 752.0| 1504.0| 734.0| 3.2705| 241800.0| NEAR BAY|
 -122.26| 37.85| 52.0| 2491.0| 474.0| 1098.0| 468.0| 3.075| 213500.0| NEAR BAY|
 -122.26| 37.84| 52.0| 696.0| 191.0| 345.0| 174.0| 2.6736| 191300.0| NEAR BAY|
 -122.26| 37.85| 52.0| 2643.0| 626.0| 1212.0| 620.0| 1.9167| 159200.0| NEAR BAY|
 -122.26| 37.85| 50.0| 1120.0| 283.0| 697.0| 264.0| 2.125| 140000.0| NEAR BAY|
 -122.27| 37.85| 52.0| 1966.0| 347.0| 793.0| 331.0| 2.775| 152500.0| NEAR BAY|
 -122.27| 37.85| 52.0| 1228.0| 293.0| 648.0| 303.0| 2.1202| 155500.0| NEAR BAY|
 -122.26| 37.84| 50.0| 2239.0| 455.0| 990.0| 419.0| 1.9911| 158700.0| NEAR BAY|
 -122.27| 37.84| 52.0| 1503.0| 298.0| 690.0| 275.0| 2.6033| 162900.0| NEAR BAY|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
only showing top 20 rows

In [5]:
df = data.toPandas()

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [6]:
df = pd.get_dummies(df, columns=["ocean_proximity"])

In [7]:
df = df.dropna(axis="rows")

In [8]:
dim = df.drop("median_house_value", axis="columns").shape[1]

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(8, input_dim=dim, activation="relu"),
  tf.keras.layers.Dense(4, activation="relu"),
  tf.keras.layers.Dense(2, activation="relu"),
  tf.keras.layers.Dense(1, activation="linear")
])

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.

In [9]:
model.compile(optimizer='rmsprop', loss='mean_absolute_percentage_error')

In [10]:
model.fit(df.drop("median_house_value", axis="columns"), df["median_house_value"], epochs=20)

Epoch 1/20
 32/20433 [..............................] - ETA: 5:58 - loss: 99.9943 832/20433 [>.............................] - ETA: 14s - loss: 99.9940  1696/20433 [=>............................] - ETA: 7s - loss: 99.9941  2464/20433 [==>...........................] - ETA: 5s - loss: 99.9942 3200/20433 [===>..........................] - ETA: 4s - loss: 99.9941 4032/20433 [====>.........................] - ETA: 3s - loss: 99.9941 4768/20433 [======>.......................] - ETA: 2s - loss: 99.9941 5568/20433 [=======>......................] - ETA: 2s - loss: 99.9941 6432/20433 [========>.....................] - ETA: 2s - loss: 99.9941 7136/20433 [=========>....................] - ETA: 1s - loss: 99.9941 7968/20433 [==========>...................] - ETA: 1s - loss: 99.9940 8736/20433 [===========>..................] - ETA: 1s - loss: 99.9941 9568/20433 [=============>................] - ETA: 1s - loss: 99.994110432/20433 [==============>...............] - ETA: 1s - loss: 99.994111264/20433 [===============>..............] - ETA: 1s - loss: 99.994012160/20433 [================>.............] - ETA: 0s - loss: 99.994013056/20433 [==================>...........] - ETA: 0s - loss: 99.994013920/20433 [===================>..........] - ETA: 0s - loss: 99.994014816/20433 [====================>.........] - ETA: 0s - loss: 99.994015648/20433 [=====================>........] - ETA: 0s - loss: 99.994016480/20433 [=======================>......] - ETA: 0s - loss: 99.994017280/20433 [========================>.....] - ETA: 0s - loss: 99.994018112/20433 [=========================>....] - ETA: 0s - loss: 99.994018912/20433 [==========================>...] - ETA: 0s - loss: 99.994019808/20433 [============================>.] - ETA: 0s - loss: 99.994020433/20433 [==============================] - 2s 89us/sample - loss: 99.9940
Epoch 2/20
 32/20433 [..............................] - ETA: 1s - loss: 99.9936 768/20433 [>.............................] - ETA: 1s - loss: 99.9939 1600/20433 [=>............................] - ETA: 1s - loss: 99.9938